# Preliminary Data Exploration and Data Cleaning with Pandas

Now that we've got our data loaded into our dataframe, we need to take a closer look at it to help us understand what it is we are working with. This is always the first step with any data science project. Let's see if we can answer the following questions: 

- How many rows does our dataframe have? 
- How many columns does it have?
- What are the labels for the columns? Do the columns have names?
- Are there any missing values in our dataframe? Does our dataframe contain any bad data?

We can use the ```.head()``` method to peek at the top 5 rows of our dataframe. To see the number of rows and columns we can use the shape attribute:

```df.shape``` 

Do you see 51 rows and 6 columns printed out below the cell? 

We saw that each column had a name. We can access the column names directly with the ```columns``` attribute.

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/1.png)


## Missing Values and Junk Data

Before we can proceed with our analysis we should try and figure out if there are any missing or junk data in our dataframe. That way we can avoid problems later on. In this case, we're going to look for NaN (Not A Number) values in our dataframe. NAN values are blank cells or cells that contain strings instead of numbers. Use the ```.isna()``` method and see if you can spot if there's a problem somewhere.

```df.isna()```

Did you find anything? Check the last couple of rows in the dataframe:

```df.tail()```

Aha! We have a row that contains some information regarding the source of the data with blank values for all the other other columns.

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/2.png)

Delete the Last Row

We don't want this row in our dataframe. There's two ways you can go about removing this row. The first way is to manually remove the row at index 50. The second way is to simply use the ```.dropna()``` method from pandas. Let's create a new dataframe without the last row and examine the last 5 rows to make sure we removed the last row:

```clean_df = df.dropna()```

```clean_df.tail()```


## Accessing Columns and Individual Cells in a Dataframe

#### Find College Major with Highest Starting Salaries

To access a particular column from a data frame we can use the square bracket notation, like so:

```clean_df['Starting Median Salary']```

You should see all the values printed out below the cell for just this column:

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/3.png)

To find the highest starting salary we can simply chain the ```.max()``` method.

```clean_df['Starting Median Salary'].max()```

The highest starting salary is $74,300. But which college major earns this much on average? For this, we need to know the row number or index so that we can look up the name of the major. Lucky for us, the ```.idxmax()``` method will give us index for the row with the largest value.

```clean_df['Starting Median Salary'].idxmax()```

which is 43. To see the name of the major that corresponds to that particular row, we can use the ```.loc``` (location) property.

```clean_df['Undergraduate Major'].loc[43]```

Here we are selecting both a column ('Undergraduate Major') and a row at index 43, so we are retrieving the value of a particular cell. You might see people using the double square brackets notation to achieve exactly the same thing: 

```clean_df['Undergraduate Major'][43]```

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/4.png)

If you don't specify a particular column you can use the ```.loc``` property to retrieve an entire row:

```clean_df.loc[43]```

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/5.png)

## Challenge

Now that we've found the major with the highest starting salary, can you write the code to find the following:

- What college major has the highest mid-career salary? How much do graduates with this major earn? (Mid-career is defined as having 10+ years of experience).
- Which college major has the lowest starting salary and how much do graduates earn after university?
- Which college major has the lowest mid-career salary and how much can people expect to earn with this degree? 

## Sorting Values & Adding Columns: Majors with the Most Potential vs Lowest Risk

Lowest Risk Majors

A low-risk major is a degree where there is a small difference between the lowest and highest salaries. In other words, if the difference between the 10th percentile and the 90th percentile earnings of your major is small, then you can be more certain about your salary after you graduate.

How would we calculate the difference between the earnings of the 10th and 90th percentile? Well, Pandas allows us to do simple arithmetic with entire columns, so all we need to do is take the difference between the two columns:

```clean_df['Mid-Career 90th Percentile Salary'] - clean_df['Mid-Career 10th Percentile Salary']```

Alternatively, you can also use the ```.subtract()``` method.

```clean_df['Mid-Career 90th Percentile Salary'].subtract(clean_df['Mid-Career 10th Percentile Salary'])```


The output of this computation will be another Pandas dataframe column. We can add this to our existing dataframe with the ```.insert()``` method:

```spread_col = clean_df['Mid-Career 90th Percentile Salary'] - clean_df['Mid-Career 10th Percentile Salary']```

```clean_df.insert(1, 'Spread', spread_col)```

```clean_df.head()```

The first argument is the position of where the column should be inserted. In our case, it's at position 1, so the second column.

![Screenshot Output](https://raw.githubusercontent.com/GermanPaul12/STADS-Python-Course/main/Project%208%20Data%20Analysis/Salaries%20by%20college%20major/assets/6.png)

#### Sorting by the Lowest Spread

To see which degrees have the smallest spread, we can use the ```.sort_values()``` method. And since we are interested in only seeing the name of the degree and the major, we can pass a list of these two column names to look at the ```.head()``` of these two columns exclusively.

```low_risk = clean_df.sort_values('Spread')```

```low_risk[['Undergraduate Major', 'Spread']].head()```

Does ```.sort_values()``` sort in ascending or descending order? To find out, check out the Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

You can also bring up the quick documentation with shift + tab on your keyboard directly in the Python notebook.

## Challenge

Using the .sort_values() method, can you find the degrees with the highest potential? Find the top 5 degrees with the highest values in the 90th percentile. 

Also, find the degrees with the greatest spread in salaries. Which majors have the largest difference between high and low earners after graduation.



## Grouping and Pivoting Data with Pandas

Often times you will want to sum rows that belong to a particular category. For example, which category of degrees has the highest average salary? Is it STEM, Business or HASS (Humanities, Arts, and Social Science)? 

To answer this question we need to learn to use the ```.groupby()``` method. This allows us to manipulate data similar to a Microsoft Excel Pivot Table.

We have three categories in the 'Group' column: STEM, HASS and Business. Let's count how many majors we have in each category:

```clean_df.groupby('Group').count()```

## Mini Challenge

Now can you use the ```.mean()``` method to find the average salary by group? 

Number formats in the Output

The above is a little hard to read, isn't it? We can tell Pandas to print the numbers in our notebook to look like 1,012.45 with the following line:

```pd.options.display.float_format = '{:,.2f}'.format```
 
Ah, that's better, isn't it?

The PayScale dataset used in this lesson was from 2008 and looked at the prior 10 years. Notice how Finance ranked very high on post-degree earnings at the time. However, we all know there was a massive financial crash in that year. Perhaps things have changed.